In [1]:
import mitiq
mitiq.SUPPORTED_PROGRAM_TYPES.keys()

dict_keys(['cirq', 'pyquil', 'qiskit', 'braket', 'pennylane', 'qibo'])

In [2]:
#select the frontend from above dict_keys
frontend = "qiskit"
frontend = "cirq"

In [3]:
from mitiq import benchmarks

# single-qubit randomized-benchmarking circuit, identity operation.

circuit0 = benchmarks.generate_rb_circuits(n_qubits=1, num_cliffords=2, return_type=frontend,)[0] # identity operation

# generate_rb_circuits :generate randomized-benchmarking circuit.
# num_cliffords : number of random Clifford group elements, proportional to the depth of the circuit.
print("Circuit0:\n")
print(circuit0)

Circuit0:

0: ───Y^-0.5───X^0.5───Y^0.5───X^0.5───Y^-0.5───X^-0.5───


In [4]:
# creating circuit using cirq backend

import cirq
"""Creating a circuit."""

# Define three qubits.
a = cirq.q("a")
b = cirq.q("b")
c = cirq.q("c")

# Define four qubit list
q = cirq.LineQubit.range(4)


# Define a list of operations.
ops1 = [cirq.H(a), cirq.H(b), cirq.CNOT(b, c), cirq.H(b)]
ops2 = [cirq.CNOT(a,c), cirq.H(b), cirq.CNOT(b, c), cirq.H(b)]
ops3 = [cirq.H(q[0]), cirq.H(q[1]), cirq.CNOT(q[1], q[2]), cirq.CNOT(q[0], q[3]), cirq.H(q[1])]

# Create a circuit from the list of operations.
circuit1 = cirq.Circuit(ops1)
circuit2 = cirq.Circuit(ops2)
circuit3 = cirq.Circuit(ops3)

print("Circuit1:\n")
print(circuit1)
print("Circuit2:\n")
print(circuit2)
print("Circuit3:\n")
print(circuit3)

Circuit1:

a: ───H───────────

b: ───H───@───H───
          │
c: ───────X───────
Circuit2:

      ┌──┐
a: ────@─────────────
       │
b: ────┼H────@───H───
       │     │
c: ────X─────X───────
      └──┘
Circuit3:

          ┌──┐
0: ───H─────@────────
            │
1: ───H────@┼────H───
           ││
2: ────────X┼────────
            │
3: ─────────X────────
          └──┘


In [5]:
# creating circuit using qiskit frontend

import qiskit
from qiskit import QuantumCircuit,QuantumRegister

circuit4=QuantumCircuit(2)
circuit4.h(0)
circuit4.cx(0,1)
circuit4.draw("mpl")

circuit5=QuantumCircuit(3)
circuit5.h(0)
circuit5.cx(0,1)
circuit5.cx(0,2)
circuit5.cx(1,0)
circuit5.draw("mpl")
print("circuit4:\n", circuit4, "\n circuit5:\n" ,circuit5)

circuit4:
      ┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘ 
 circuit5:
      ┌───┐          ┌───┐
q_0: ┤ H ├──■────■──┤ X ├
     └───┘┌─┴─┐  │  └─┬─┘
q_1: ─────┤ X ├──┼────■──
          └───┘┌─┴─┐     
q_2: ──────────┤ X ├─────
               └───┘     


/home/a/miniconda3/envs/mitiq/lib/python3.11/site-packages/qiskit/visualization/circuit/matplotlib.py:266: FutureWarning: The default matplotlib drawer scheme will be changed to "iqp" in a following release. To silence this warning, specify the current default explicitly as style="clifford", or the new default as style="iqp".
  self._style, def_font_ratio = load_style(self._style)


In [26]:
# define an executor which simulates a backend with depolarizing noise and computes probability of the 00 state.
# first convert input circuit to the Mitiq internal representation (Cirq) and then simulate it on simulator

from cirq import DensityMatrixSimulator, depolarize # selecting simulator and noise
from mitiq.interface import convert_to_mitiq

def execute(circuit, noise_level=0.5):
    """Returns Tr[ρ |0⟩⟨0|] (expectation value)where ρ is the state prepared by the circuit
    executed with depolarizing noise.
    """
    # Replace with code based on your frontend and backend.
    mitiq_circuit, _ = convert_to_mitiq(circuit)
    
    # We add a simple noise model to simulate a noisy backend, depolarizing noise
    noisy_circuit = mitiq_circuit.with_noise(depolarize(p=noise_level))
    #cirq.depolarize( p: float, n_qubits: int = 1): depolarize noise, p = noise or probability that one of the Pauli gates is applied
    
    rho = DensityMatrixSimulator().simulate(noisy_circuit).final_density_matrix
    #final_density_matrix(program,*,initial_state,qubit_order):Returns the density matrix resulting from simulating the circuit
    #DensityMatrixSimulator(*,dtype,noise):simulator for density matrices and noisy quantum circuits
    return rho[0, 0].real #real value of matrix element (0,0)
#single qubit depolarizing noise works on single qubit only?
#how working on multiqubit circuit?

In [7]:
mitiq_circuit, _ = convert_to_mitiq(circuit1)
    
    # We add a simple noise model to simulate a noisy backend, depolarizing noise
noisy_circuit = mitiq_circuit.with_noise(depolarize(p=0.2))
    #cirq.depolarize( p: float, n_qubits: int = 1): depolarize noise, p = noise or probability that one of the Pauli gates is applied
    
rho = DensityMatrixSimulator().simulate(noisy_circuit).final_density_matrix
print(rho)
print(rho[1,1])
print(rho[1,1].real)

[[ 0.12499998+0.j  0.02651045+0.j  0.02651044+0.j -0.019441  +0.j
   0.04929629+0.j  0.01045494+0.j  0.01045493+0.j -0.00766695+0.j]
 [ 0.02651045+0.j  0.12499998+0.j  0.019441  +0.j -0.02651044+0.j
   0.01045494+0.j  0.04929629+0.j  0.00766695+0.j -0.01045493+0.j]
 [ 0.02651044+0.j  0.019441  +0.j  0.12499998+0.j -0.02651045+0.j
   0.01045493+0.j  0.00766695+0.j  0.04929629+0.j -0.01045494+0.j]
 [-0.019441  +0.j -0.02651045+0.j -0.02651045+0.j  0.12499998+0.j
  -0.00766695+0.j -0.01045494+0.j -0.01045494+0.j  0.04929629+0.j]
 [ 0.04929629+0.j  0.01045494+0.j  0.01045493+0.j -0.00766695+0.j
   0.12499998+0.j  0.02651045+0.j  0.02651044+0.j -0.019441  +0.j]
 [ 0.01045494+0.j  0.04929629+0.j  0.00766695+0.j -0.01045493+0.j
   0.02651045+0.j  0.12499998+0.j  0.019441  +0.j -0.02651044+0.j]
 [ 0.01045493+0.j  0.00766695+0.j  0.04929629+0.j -0.01045494+0.j
   0.02651044+0.j  0.019441  +0.j  0.12499998+0.j -0.02651045+0.j]
 [-0.00766695+0.j -0.01045494+0.j -0.01045494+0.j  0.04929629+0.j
  -

Depolarizing channel on one qubit:
$$ \rho \rightarrow (1-p) \rho + \frac{p}{3} (X \rho X + Y \rho Y + Z \rho Z) $$
where the, $$ A_k $$ are Kraus operators. These operators are not necessarily unitary and satisfy the property
$$ \sum_k A_k^\dagger A_k = I $$

Cirq currently supports noise that fits within the context of operator sum representations of noise (a.k.a quantum operations, quantum dyanamical maps, superoperators, etc). This formalism models the evolution of a density matrix via

$$ \rho \rightarrow \sum_k A_k \rho A_k^\dagger $$


This channel applies one of $ 4^n $disjoint possibilities: nothing (the identity channel) or one of the $ (4^n - 1) $ pauli gates. The disjoint probabilities of the non-identity Pauli gates are all the same, $ p / (4^n - 1)$ and the identity is done with probability 
$ 1 - p $. The supplied probability must be a valid probability or else this constructor will raise a ValueError.

This channel evolves a density matrix via
$$ \rho \rightarrow (1 - p) \rho + p / (4^n - 1) \sum _i P_i \rho P_i $$

where $ P_i $ are the $ (4^n - 1) $ Pauli gates (excluding the identity).

In [27]:
## function execute is used to evaluate noisy (unmitigated) expectation values.
#Compute the expectation value of the |0⟩⟨0| observable.
noisy_value0= execute(circuit0)
ideal_value0= execute(circuit0, noise_level=0.0)
print(f"For Circuit0: Error without mitigation: {abs(ideal_value0- noisy_value0):5f}")
noisy_value1= execute(circuit1)
ideal_value1= execute(circuit1, noise_level=0.0)
print(f"For Circuit1: Error without mitigation: {abs(ideal_value1- noisy_value1):.15f}")
noisy_value2= execute(circuit2)
ideal_value2= execute(circuit2, noise_level=0.0)
print(f"For Circuit2: Error without mitigation: {abs(ideal_value2- noisy_value2):.10f}")
noisy_value3= execute(circuit3)
ideal_value3= execute(circuit3, noise_level=0.0)
print(f"For Circuit3: Error without mitigation: {abs(ideal_value3- noisy_value3):.5f}")
noisy_value4= execute(circuit4)
ideal_value4= execute(circuit4, noise_level=0.0)
print(f"For Circuit4:Error without mitigation: {abs(ideal_value4- noisy_value4):.5f}")
noisy_value5= execute(circuit5)
ideal_value5= execute(circuit5, noise_level=0.0)
print(f"For Circuit5:Error without mitigation: {abs(ideal_value5- noisy_value5):.5f}")

For Circuit0: Error without mitigation: 0.499314
For Circuit1: Error without mitigation: 0.000000022351742
For Circuit2: Error without mitigation: 0.1203703284
For Circuit3: Error without mitigation: 0.06224
For Circuit4:Error without mitigation: 0.24074
For Circuit5:Error without mitigation: 0.37476


In [28]:
# the execute function simulates the effect of depolarizing noise of some fixed noise_level acting after each gate
from mitiq.pec.representations.depolarizing import represent_operations_in_circuit_with_local_depolarizing_noise

noise_level = 0.5 #same as defined in execute() function
reps10 = represent_operations_in_circuit_with_local_depolarizing_noise(circuit1, noise_level)
print(f"{len(reps10)} OperationRepresentation objects produced, assuming {100 * noise_level}% depolarizing noise.")

3 OperationRepresentation objects produced, assuming 50.0% depolarizing noise.


In [25]:
#showing how an ideal gate can be expressed as a linear combination of noisy gates
#reps[0],argumentfor diffrent number of qubit
print(reps10[0]) 
print(reps10[1])
print(reps10[2])
print(1.115-0.038-0.038-0.038)
print(1.244-0.043-0.043-0.043-0.043-0.043+9*0.001)

b: ───H─── = 1.115*(b: ───H───)-0.038*(b: ───H───X───)-0.038*(b: ───H───Y───)-0.038*(b: ───H───Z───)
a: ───H─── = 1.115*(a: ───H───)-0.038*(a: ───H───X───)-0.038*(a: ───H───Y───)-0.038*(a: ───H───Z───)
b: ───@───
      │
c: ───X─── = 

1.244
b: ───@───
      │
c: ───X───

-0.043
b: ───@───────
      │
c: ───X───X───

-0.043
b: ───@───────
      │
c: ───X───Y───

-0.043
b: ───@───────
      │
c: ───X───Z───

-0.043
b: ───@───X───
      │
c: ───X───────

-0.043
b: ───@───Y───
      │
c: ───X───────

-0.043
b: ───@───Z───
      │
c: ───X───────

+0.001
b: ───@───X───
      │
c: ───X───X───

+0.001
b: ───@───Y───
      │
c: ───X───X───

+0.001
b: ───@───Z───
      │
c: ───X───X───

+0.001
b: ───@───X───
      │
c: ───X───Y───

+0.001
b: ───@───Y───
      │
c: ───X───Y───

+0.001
b: ───@───Z───
      │
c: ───X───Y───

+0.001
b: ───@───X───
      │
c: ───X───Z───

+0.001
b: ───@───Y───
      │
c: ───X───Z───

+0.001
b: ───@───Z───
      │
c: ───X───Z───
1.001
1.0380000000000003


In [22]:
noise_level = 0.1 #same as defined in execute() function
reps11 = represent_operations_in_circuit_with_local_depolarizing_noise(circuit3, noise_level)
print(f"{len(reps11)} OperationRepresentation objects produced, assuming {100 * noise_level}% depolarizing noise.")
print(reps11[0]) 
print(reps11[1])
print(reps11[2])
print(reps11[3])

4 OperationRepresentation objects produced, assuming 10.0% depolarizing noise.
0: ───H─── = 1.115*(0: ───H───)-0.038*(0: ───H───X───)-0.038*(0: ───H───Y───)-0.038*(0: ───H───Z───)
1: ───H─── = 1.115*(1: ───H───)-0.038*(1: ───H───X───)-0.038*(1: ───H───Y───)-0.038*(1: ───H───Z───)
0: ───@───
      │
3: ───X─── = 

1.244
0: ───@───
      │
3: ───X───

-0.043
0: ───@───X───
      │
3: ───X───────

-0.043
0: ───@───Y───
      │
3: ───X───────

-0.043
0: ───@───Z───
      │
3: ───X───────

-0.043
0: ───@───────
      │
3: ───X───X───

-0.043
0: ───@───────
      │
3: ───X───Y───

-0.043
0: ───@───────
      │
3: ───X───Z───

+0.001
0: ───@───X───
      │
3: ───X───X───

+0.001
0: ───@───X───
      │
3: ───X───Y───

+0.001
0: ───@───X───
      │
3: ───X───Z───

+0.001
0: ───@───Y───
      │
3: ───X───X───

+0.001
0: ───@───Y───
      │
3: ───X───Y───

+0.001
0: ───@───Y───
      │
3: ───X───Z───

+0.001
0: ───@───Z───
      │
3: ───X───X───

+0.001
0: ───@───Z───
      │
3: ───X───Y───

+0.0

In [23]:
reps0 = represent_operations_in_circuit_with_local_depolarizing_noise(circuit0, noise_level)
reps1 = represent_operations_in_circuit_with_local_depolarizing_noise(circuit1, noise_level)
reps2 = represent_operations_in_circuit_with_local_depolarizing_noise(circuit2, noise_level)
reps3 = represent_operations_in_circuit_with_local_depolarizing_noise(circuit3, noise_level)
reps4 = represent_operations_in_circuit_with_local_depolarizing_noise(circuit4, noise_level)
reps5 = represent_operations_in_circuit_with_local_depolarizing_noise(circuit5, noise_level)

In [29]:
#Applying PEC

# For PEC to work properly, the noise model and the noise_level associated to the execute function must correspond to those used to compute the OperationRepresentation objects.
# Once the necessary OperationRepresentations have been defined, probabilistic error cancellation can be applied through the execute_with_pec() function as follows:
from mitiq import pec

pec_value0 = pec.execute_with_pec(circuit0, execute, representations=reps0)
print(f"Error without PEC:Circuit0: {abs(ideal_value0 - noisy_value0) :.5f}")
print(f"Error with PEC:Circuit0:    {abs(ideal_value0 - pec_value0) :.5f}")

pec_value1 = pec.execute_with_pec(circuit1, execute, representations=reps1)
print(f"Error without PEC:Circuit1: {abs(ideal_value1 - noisy_value1) :.15f}")
print(f"Error with PEC:Circuit1:   {abs(ideal_value1 - pec_value1) :.15f}")

pec_value2 = pec.execute_with_pec(circuit2, execute, representations=reps2)
print(f"Error without PEC:Circuit2: {abs(ideal_value2 - noisy_value2) :.5f}")
print(f"Error with PEC:Circuit2:    {abs(ideal_value2 - pec_value2) :.5f}")

pec_value3 = pec.execute_with_pec(circuit3, execute, representations=reps3)
print(f"Error without PEC:Circuit3: {abs(ideal_value3 - noisy_value3) :.5f}")
print(f"Error with PEC:Circuit3:    {abs(ideal_value3 - pec_value3) :.5f}")

pec_value4 = pec.execute_with_pec(circuit4, execute, representations=reps4)
print(f"Error without PEC:Circuit4: {abs(ideal_value4 - noisy_value4) :.5f}")
print(f"Error with PEC:Circuit4:    {abs(ideal_value4 - pec_value4) :.5f}")

pec_value5 = pec.execute_with_pec(circuit5, execute, representations=reps5)
print(f"Error without PEC:Circuit5: {abs(ideal_value5 - noisy_value5) :.5f}")
print(f"Error with PEC:Circuit5:    {abs(ideal_value5 - pec_value5) :.5f}")

Error without PEC:Circuit0: 0.49931
Error with PEC:Circuit0:    0.47355
Error without PEC:Circuit1: 0.000000022351742
Error with PEC:Circuit1:   0.006270131734814
Error without PEC:Circuit2: 0.12037
Error with PEC:Circuit2:    0.11809
Error without PEC:Circuit3: 0.06224
Error with PEC:Circuit3:    0.05841
Error without PEC:Circuit4: 0.24074
Error with PEC:Circuit4:    0.23713
Error without PEC:Circuit5: 0.37476
Error with PEC:Circuit5:    0.37154


In [ ]:
# how to configute git
#use this command in terminal

#git configuration
git #to check git is installed or not
sudo apt install git # if git is not installed
git --version #to check git version

# creating ssh key
ssh-keygen -t ed25519 -C "your_email" #it create public and private key file
cat /home/a/.ssh/id_ed25519.pub # to access the public key
# /home/a/.ssh/id_ed25519.pub is location of saved public key
#copy the complete code line and paste it to generate ssh key in git account
 
ssh -T git@github.com #test

#create a folder for new project,PEC, navigate to this folder
git clone https://github.com/gachchami/qmitra.git # to clone the repository
#again navigate to main downloaded folder
git status # to check status
# copy ur file to the folder (main file)
git checkout PEC
git diff
git config --global user.email "anand89694@gmail.com"
git commit -m 'updated'
rm /home/a/PEC/qmitra/.git/index.lock
git commit -a -m "updated"
git help commit
git push
git config --global url."ssh://git@github.com".insteadOf "https://github.com"
git push 



#initializing empty git repository
git init # create a folder named .get ,track changes
git status# for checking status of repo
# red colour file is not added yet in repo
git add filename.py #add filename.py file to repo
git add . # add all the file to repo
git commit -m "message" # if branch is not mentioned, default to main
git branch -M nameofbranch 
git remote add origin gitrepolink  # connect local to git repo
git push -u origin main # or write name of branch
git status
git comit - m "erge"
git push